In [ ]:
import pandas as pd
import numpy as np
import googlemaps
import time
import os.path
import json
import datetime
import warnings
warnings.filterwarnings('ignore')
import polyline

In [ ]:
## Courtesy Andy Eschbacher - a part of his cool project Cartoframes!
def to_carto(df, tablename=None, username=None, api_key=None):
    import requests
    import json
    if tablename is None:
        tablename = 'testing_upload'
    tablename = ('%s.csv' % tablename)
    df.to_csv(tablename)
    files = {'file': open(tablename)}
    post_template = 'https://{username}.carto.com/api/v1/imports/?api_key={api_key}'
    try:
        r = requests.post(post_template.format(
                username=username,
                api_key=api_key), files=files)
    except Exception, err:
        print("Error: %s" % err)
    if json.loads(r.content)['success']:
        print('File successfully uploaded')
    else:
        print('File failed to upload')
    return json.loads(r.content)['success']

### Setting up a CARTO query template

In [ ]:
carto_key = 'Insert your CARTO API key here'
username = 'Insert your username here'
#Please do not leave any spaces or it will fail!

#Insert your CARTO API key i.e. -> carto_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [ ]:
query_template = 'http://{username}.cartodb.com/api/v2/sql?q={query}&format=csv&api_key={api_key}'
query = "SELECT * FROM citibike_mar17".replace(' ', '%20')

citibike = pd.read_csv(query_template.format(username=username, api_key=carto_key, query=query))

### Randomly selecting one bike and one day

In [ ]:
random = "SELECT trip_data.cartodb_id, trip_data.bike_id, trip_data.start_time, trip_data.stop_time, trip_data.trip_duration, trip_data.start_station_name, trip_data.end_station_name, trip_data.start_station_latitude, trip_data.start_station_longitude, trip_data.end_station_latitude, trip_data.end_station_longitude FROM( SELECT bike_id, start_time, stop_time, start_station_name, end_station_name, trip_duration, cartodb_id, start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude FROM citibike_mar17 as a JOIN(SELECT unnest(Array[26920, 26785, 25818, 26541, 25597, 26354]) as oneid, Date(start_time) onedate FROM citibike_mar17 ORDER BY RANDOM() LIMIT 1) b ON a.bike_id = b.oneid WHERE Date(start_time) = onedate ) as trip_data ORDER BY trip_data.start_time ASC".replace(' ', '%20')
one_trip = pd.read_csv(query_template.format(query=random))
one_trip

### Google Maps Directions API

In [ ]:
gmaps = googlemaps.Client(key='Insert your Google Maps API key here')

#### `node` is our list with all routes and `durations` is our list with all durations

In [ ]:
node = list()
durations = list()

### Using Google Maps Directions API to get routes for all pairs of origins and destinations

In [ ]:
for r in range(len(one_trip)):
    origin = str(one_trip['start_station_latitude'][r]), str(one_trip['start_station_longitude'][r])
    destination = str(one_trip['end_station_latitude'][r]), str(one_trip['end_station_longitude'][r])
    start = one_trip['start_time'][r][:19]
    dt = datetime.datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
    date = datetime.datetime(dt.year, dt.month+2, dt.day,dt.hour,dt.minute,dt.second)
    directions_result = gmaps.directions(origin,
                                     destination,
                                     mode="bicycling",
                                    departure_time = date)
    node.append(polyline.decode(directions_result[0]['overview_polyline']['points']))
    durations.append(directions_result[0]['legs'][0]['duration']['value'])

In [ ]:
origin = str(one_trip['start_station_latitude'][0]), str(one_trip['start_station_longitude'][0])
destination = str(one_trip['end_station_latitude'][0]), str(one_trip['end_station_longitude'][0])
start = one_trip['start_time'][0][:19]
dt = datetime.datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
date = datetime.datetime(dt.year, dt.month+2, dt.day,dt.hour,dt.minute,dt.second)

### Making a DataFrame from our response

In [ ]:
length = 0
for r in range(len(node)):
    length = length + len(node[r])

In [ ]:
length

In [ ]:
latlon = pd.DataFrame(index=np.arange(length),columns={'lat','lon','trip_id','duration'})

In [ ]:
q = 0
c = 0
t = 0
r = 0
p = 0
stop = 0

In [ ]:
for r in range(len(node)):
    c = c + q
    for t in range(len(node[r])):
        p = c + t
        latlon['trip_id'][p] = r
        latlon['lat'][p] = node[r][t][0]
        latlon['lon'][p] = node[r][t][1]
        if (q==0):
            latlon['duration'][p] = 0
        else:
            latlon['duration'][p] = latlon['duration'][p-1] + durations[r]/len(node[r]) + stop
        q = len(node[r])
    stop = 30
    #simlulating station stop 

In [ ]:
latlon.isnull().values.any()

### Output tables

#### Load directly into CARTO

In [ ]:
to_carto(latlon, tablename = 'random_trip_route', username='Insert your username', api_key='Insert you API key')

In [ ]:
to_carto(one_trip, tablename = 'random-trip-stations', username='Insert your username', api_key='Insert your API key')

#### Or Download as .csv

In [ ]:
latlon.to_csv('random_trip_route.csv')

In [ ]:
one_trip.to_csv('random_trip_stations.csv')